# Importing the required modules

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import spacy
import warnings 
warnings.filterwarnings('ignore')

In [29]:
pip install sentence-transformers

     -------------------------------------- 79.7/79.7 kB 221.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.0/1.0 MB 1.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120735 sha256=f61beb28f0900e4ed36e348db670a579326c3b2907b52ff1135e3209ac251875Note: you may need to restart the kernel to use updated packages.

  Stored in directory: c:\users\mitug\appdata\local\pip\cache\wheels\2b\11\3b\32a18fb9f2253b25d3d1a06f0a84e2d516e7efa19c8c71a283
Successfully built sentence-transformers


# Question Input

In [46]:
qs = input()

Is 56788 the train number of R Express?


# Intent Classification

In [53]:
file = open(r"..\data\intent_classification_data.json")
data = json.load(file)

In [54]:
data

{'TrainCheck': ['Is X the train number of Y?', 'Does X have train number Y?'],
 'RouteCheck': ['Are X and Y connected by rail?',
  'Is there a train connecting X and Y?']}

In [55]:
intent_similiarity = dict()

In [56]:
def question_similiarity(sentence,question):
    sent = nlp(sentence)
    q = nlp(question)
    return sent.similarity(q)

In [58]:
for i in data:
    intent_similiarity[i] = np.mean(list(map(question_similiarity,data[i],[qs]*len(data[i]))))
print(intent_similiarity)

{'TrainCheck': 0.7433274625513855, 'RouteCheck': 0.5845521191261642}


# Entity Recognition

In [45]:
nlp = spacy.load("en_core_web_sm")

# it works with single words too
sent = nlp('Does X have train number Y?')
sent1 = nlp('Is X the train number of Y?')
sent2 = nlp('Is 56788 the train number of R Express?')
sent3 = nlp('Does Andhra Pradesh Express have train number 12723?')
sent4 = nlp('Are X and Y connected by rail?')
sent5 = nlp('Is there a train connecting X and Y?')

# similarity ranges from 0 (totally dissimilar) to 1 (identical)

print(sent1.similarity(sent2))
print(sent.similarity(sent2))
print(sent4.similarity(sent2))
print(sent5.similarity(sent2))
print(sent1.similarity(sent3))

0.8618193633531551
0.6248355617496157
0.5708127814142552
0.5982914568380734
0.5883917835046543


In [34]:
from sentence_transformers import SentenceTransformer, util
sentences = ["Is X the train number of Y?", "Is 12723 the train number of Andhra Pradesh Express?"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)
## tensor([[0.6003]])

tensor([[0.4918]])

# Question Answering

In [ ]:
tensor([[0.4986]])